# Reading Toshiba Data

In [1]:
import os
import sys
import glob
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Local modules
import netCDF4

# iRadar
sys.path.append(os.path.expanduser('~/Developer/iradar/python'))
import iradar

# Get the First File

In [2]:
files = glob.glob(os.path.expanduser('~/Downloads/toshiba/*.nc'))
filename = files[0]

azimuth_count = 360
range_count = 800

In [3]:
print(filename)

/Users/boonleng/Downloads/toshiba/20190816_115819.01-01-01-PPI0020.Ingest-ZH_MTI.AUTO-03_1.akita.dat.nc


# Read the File

In [7]:
maxgate = 2048

fid = netCDF4.Dataset(filename, 'r')

# Attributes, dimensions, and variables
atts = fid.ncattrs()
ds = [d for d in fid.dimensions]
vs = [v for v in fid.variables]
print(vs)

# Get the variables that I care about
lat = float(fid.variables['siteLat'][0])
lon = float(fid.variables['siteLon'][0])
start_time = time.gmtime(int(fid.variables['esStartTime'][0]))

scan_el = np.array(fid.variables['elevationAngle'][0], dtype=np.float)
ee = np.array(fid.variables['radialElev'])
aa = np.array(fid.variables['radialAzim'])
zz = fid.variables['Z']

r0 = float(fid.variables['firstGateRange'][0])
dr = float(fid.variables['gateSize'][0])
ng = min(maxgate, zz.shape[1])
rr = np.arange(r0, ng * dr, dr)

z = np.array(zz)[:, :maxgate] * zz.scale_factor + zz.add_offset
m = z == zz._FillValue
v = np.zeros(z.shape)
z[m] = np.nan
v[m] = np.nan

if not zz.shape[0] == len(aa):
    print('Inconsistent azimuth count!  {} / {}'.format(z.shape[0], len(aa)))

# fid.close()

['esStartTime', 'elevationNumber', 'elevationAngle', 'radialAzim', 'radialElev', 'radialTime', 'siteLat', 'siteLon', 'siteAlt', 'firstGateRange', 'gateSize', 'nyquist', 'unambigRange', 'calibConst', 'radarConst', 'beamWidthHori', 'pulseWidth', 'bandWidth', 'filterDop', 'elevationList', 'azimuthSpeed', 'highPRF', 'lowPRF', 'dwellTime', 'waveLength', 'calI0', 'calNoise', 'groundHeight', 'meltHeight', 'scanType', 'angleResolution', 'logNoise', 'linNoise', 'inphaseOffset', 'quadratureOffset', 'logSlope', 'logFilter', 'filterPntClt', 'filterThreshold', 'sampleNum', 'SQIThresh', 'LOGThresh', 'SIGThresh', 'CSRThresh', 'DBTThreshFlag', 'DBZThreshFlag', 'VELThreshFlag', 'WIDThreshFlag', 'beamWidthVert', 'Z']


# Construct a sweep dictionary

In [5]:
# o = {
#         'longitude': longitude,
#         'latitude': latitude,
#         'time': sweep_time,
#         'ee': ee,
#         'aa': aa,
#         'rr': rr,
#         'z': zs,
#         'v': vs,
#         'm': ms
#     }

sweep = {
    'longitude': lon,
    'latitude': lat,
    'time': start_time,
    'scan_el': scan_el,
    'ee': ee,
    'aa': aa,
    'rr': rr,
    'z': z,
    'v': v,
    'm': m
}

print(sweep)

{'longitude': 140.05166625976562, 'latitude': 39.74333190917969, 'time': time.struct_time(tm_year=2019, tm_mon=8, tm_mday=16, tm_hour=2, tm_min=58, tm_sec=36, tm_wday=4, tm_yday=228, tm_isdst=0), 'scan_el': array(1.99951172), 'ee': array([181.97754, 181.97754, 181.97754, 181.97754, 181.97754, 181.97754,
       181.97754, 181.97754, 181.97754, 181.97754, 181.97754, 181.97754,
       181.97754, 181.97754, 181.97754, 181.97754, 181.97754, 181.97754,
       181.97754, 181.97754, 181.97754, 181.97754, 181.97754, 181.97754,
       181.97754, 181.97754, 181.97754, 181.97754, 181.97754, 181.97754,
       181.97754, 181.97754, 181.97754, 181.97754, 181.97754, 181.97754,
       181.97754, 181.97754, 181.97754, 181.97754, 181.97754, 181.97754,
       181.97754, 181.97754, 181.97754, 181.97754, 181.97754, 181.97754,
       181.97754, 181.97754, 181.97754, 181.97754, 181.97754, 181.97754,
       181.97754, 181.97754, 181.97754, 181.97754, 181.97754, 181.97754,
       181.97754, 181.97754, 181.97754

In [9]:
q = np.array(zz)

In [12]:
np.max(q)

61.309998